In [1]:
from abc import ABC, abstractmethod

class MyAbstractClass(ABC):
    @abstractmethod
    def abstract_method(self):
        pass

    def concrete_method(self):
        print("This is a concrete method")

# Попытка создать объект абстрактного класса вызовет ошибку
# obj = MyAbstractClass()

class MySubclass(MyAbstractClass):
    def abstract_method(self):
        print("Abstract method implementation")

obj = MySubclass()
obj.abstract_method()   # Вызов абстрактного метода
obj.concrete_method()   # Вызов конкретного метода


Abstract method implementation
This is a concrete method


In [2]:
class A:
    pass

class B(A):
    pass

class C(A):
    pass

class D(B, C):
    pass

mro_result = D.mro()
print(mro_result)  


[<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>]


In [ ]:
from copy import deepcopy

class FragileDict:
    def __init__(self, d: dict = None):
        if d is None:
            d = {}
        self._data = deepcopy(d)
        self._lock = True

    def __setitem__(self, key, value) -> None:
        if self._lock:
            raise RuntimeError("Protected state")
        self._temp_data[key] = value

    def __getitem__(self, key) -> Any:
        if self._lock:
            return deepcopy(self._data[key])

        if key in self._temp_data:
            return self._temp_data[key]

        if key not in self._modified:
            self._modified[key] = deepcopy(self._data[key])

    def __contains__(self, key) -> bool:
        if self._lock:
            return key in self._data
        return key in self._temp_data

    def __enter__(self) -> 'FragileDict':
        self._lock = False
        self._temp_data = {}
        self._modified = {}
        return self

    def __exit__(self, exc_type, exc_value, traceback) -> None:
        if exc_type:
            print("Exception has been suppressed")

            for key in self._data:
                if key in self._modified:
                    self._data[key] = self._modified[key]

            del self._temp_data
            del self._modified
            self._lock = True
            return True

        for key in self._data:
            if key in self._modified:
                item = self._modified[key]
                self._data[key] = deepcopy(item)

        for key, value in self._temp_data.items():
            self._data[key] = deepcopy(value)

        del self._temp_data
        del self._modified
        self._lock = True
    

